1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import statistics
import time
from geopy.distance import geodesic
import geopy.geocoders
import re
import pickle
import google.cloud.storage

In [ ]:
# constants and shared info.

GENERATION_CONFIG = {
      "max_output_tokens": 8192,
      "temperature": 0.1,
      "top_p": 0.95,
  }

SAFETY_SETTING = {
      generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
  }

PATTERN_FOR_MID_MMI = r'"mid_mmi_value_numeric": "([0-9.]+)"'

EVENT_INFOR_DIRECTORIES = {
            "ok_2024":{"mag": 5.1, "lat_lng":(35.534, -96.764), "dir":"ok_m5.1", "event_id":"ok2024cish"},
            "nj_2024":{"mag": 4.8, "lat_lng":(40.696, -74.760), "dir":"nj_m4.8", "event_id":"us7000ma74"},
            "tw_2024":{"mag": 7.4, "lat_lng":(23.832, 121.604), "dir":"tw_m7.4", "event_id":"us7000m9g4"},
            "nc_2023":{"mag": 5.1, "lat_lng":(40.204, -121.110), "dir":"nca_m5.5", "event_id":"nc73886731"},
            "jp_2024":{"mag": 7.5, "lat_lng":(37.487, 137.271), "dir":"jp_m7.5", "event_id":"us6000m0xl"},
            "jp_2011":{"mag": 9.1, "lat_lng":(38.297, 142.373), "dir":"jp_m9.1", "event_id":"official20110311054624120"},
            "sc_2024":{"mag": 4.6, "lat_lng":(34.058, 118.908), "dir":"ca_malibu_4.6", "event_id":"ci40664632"},
            }

DATASET_PATH = "quake_gemini_project/gemini_project_dataset"


In [ ]:
# Helpper functions.

def _query_post_location(input_data: Part):
  """
  Queries the model to extract the location of the post.
  """

  text1 = """Analyze the provided image/video and extract any information that indicates the location of the user or post. Identify the specific location name, including city, state (if applicable), and country. Make sure to limit your response only to the extracted location.

  Example output: Imperial, CA"""

  vertexai.init(project="google.com:quake-eval", location="us-central1")
  model = GenerativeModel("gemini-1.5-pro-preview-0409")

  return model.generate_content(
      ["""This is the input data:""", input_data, text1],
      generation_config=GENERATION_CONFIG,
      safety_settings=SAFETY_SETTING,
      stream=True,
  )


def _calculate_distance(coord1, coord2):
  """
  Calculates the direct distance between two geographical coordinates in kilometers.

  Args:
    coord1: A tuple (latitude, longitude) representing the first location.
    coord2: A tuple (latitude, longitude) representing the second location.

  Returns:
    The distance between the two locations in kilometers.
  """
  distance = geodesic(coord1, coord2).km
  return distance

In [ ]:
## semi-automated part (needs manual checking of the extracted location and distance computation.)

event = "nj_2024"
file_number = "017"
file_type = "mp4"

earthquake_id = event_info[event]["event_id"]
post_id = f"{earthquake_id}_{file_number}.{file_type}"
earthquake_mag = event_info[event]["mag"]
earthquake_lat_lng = event_info[event]["lat_lng"]
event_direcory = event_info[event]["dir"]

if post_id.split(".")[1] == "mp4":
  input_data = Part.from_uri(
      mime_type="video/mp4",
      uri=f"gs://{DATASET_PATH}/{event_direcory}/{post_id}")
elif post_id.split(".")[1] == "jpeg":
  input_data = Part.from_uri(
      mime_type="image/jpeg",
      uri=f"gs://{DATASET_PATH}/{event_direcory}/{post_id}")

print("Earthquake: ",earthquake_id, "( M", earthquake_mag, ")")
print("Post ID: ", post_id)

Earthquake:  us7000ma74 ( M 4.8 )
Post ID:  us7000ma74_017.mp4


In [ ]:
# extracting the evnet
responses = _query_post_location(input_data)
city_state_name = "".join((response.text for response in responses))
print(city_state_name)

High Bridge, New Jersey


In [ ]:
# If it couldn't extract the location from the post content but it is known from other sources we can set it up manually, so its distance to epicenter can be computed.

#city_state_name = "Whitehall Township"

# Create a Nominatim geocoder.
geolocator = geopy.geocoders.Nominatim(user_agent="quake-gemini")
# Get the coordinates for the city.
location = geolocator.geocode(city_state_name)
# Print the coordinates.
print("City's coordinate:", location.latitude,",", location.longitude)
# Example usage
coord1 = (EARTHAUKE_LAT_LNG[0], EARTHAUKE_LAT_LNG[1])  # earthquake coodrinate
coord2 = (location.latitude, location.longitude) # post approximate coordinates
#coord2 = (40.849268, -73.897141)

distance_km = round(_calculate_distance(coord1, coord2), 2)
print(f"{CITY_STATE_NAME} is: {distance_km:.2f} km")

City's coordinate: 40.6670454 , -74.8957231


NameError: name '_calculate_distance' is not defined

In [ ]:
def get_response(input_data, distance_to_epicentr_km, earthquake_magnitude):
  time.sleep(2)

  text0 = f"""Here is the input data that a user, at {distance_to_epicentr_km} km from the epicenter of a M{earthquake_magnitude} earthquake, reported: """

  text1 = """You are an earthquake seismologist. Considering the popularity of YouTube and social media platforms like Twitter and TikTok, you intend to use the multimedia content in social media posts shared by people who experienced an earthquake to estimate the intensity of earthquake ground shaking, in the Modified Mercalli Intensity (MMI) Scale, at their locations. Please estimate the intensity of earthquake ground shaking using the provided examples as a reference. Moreover, classify it into two classes of: a) CCTV Footage or b) Social-Media Post/News Interview. Make sure to limit your response to a JSON format containing only the following keys: "felt_by_user", "class_source", "post_location", "post_date_time", "earthquake_location", "earthquake_magnitude", "distance_to_earthquake_epicenter", "shaking_duration", "building_type",  "building_materials", "human_reaction","animals_reaction", "furnishing", "language", "natural_environment", "video_evidence", "audio_evidence", "textual_evidence", "mmi_estimation", "mid_mmi_value_numeric","estimation_confidence", "visual_observation", "auditory_cues", "textual_information", "analysis_of_evidences", "reasoning". Let's walk this through step by step.

  Example 1:
  Input:
  This is an example of input Tweets, posted by a user who felt the M 4.6 earthquake, 3 miles from the epicenter:

  Output:
  Available Data:
  Class Source: social-media post/news interviews (Twitter)
  Felt by User: No
  Post’s Location: Imperial, CA
  Post’s Date and Time: February 12, 2:57 AM
  Earthquake\'s Location: Not identifiable from post content.
  Earthquake\'s Magnitude: 4.6 (provided in prompt)
  Distance to Earthquake Epicenter: 2 km (provided in prompt)
  Shaking Duration: Not visible in the content.
  Building Type: Residential building.
  Building Materials: Wooden Framed.
  Human\'s Reaction: Not visible in the content.
  Animal\'s Reaction: Not visible in the content.
  Furnishings:  Hanging light fixture (chandelier or pendant light), table and chairs, cabinets, and other typical kitchen items.
  Natural Environment: Not visible.

  Textual Evidence:
  Hashtag #earthquake: The user\'s inclusion of \"#earthquake\" explicitly indicates they experienced an earthquake.

  MMI Scale Estimation:
  Based on the available evidence, the earthquake intensity at this location is likely within the M range:
  MMI IV (Light): During the day felt indoors by many, outdoors by few. At night some awakened. Dishes, windows, and doors disturbed; walls make cracking sounds. Sensation like a heavy truck striking building. Standing motor cars rocked noticeably.
  MMI V (Moderate): Felt by nearly everyone; many awakened. Some Dishes, windows broken. Unstable objects overturned. Pendulum clocks may stop.

  Reasoning:
  Visual Observations:
  Chandelier Swinging: The prominent movement of the chandelier is a clear indicator of ground shaking. Its swinging motion suggests a level of intensity consistent with MMI IV (felt indoors by many, outdoors by few; dishes, windows, doors disturbed; walls make cracking sound) or MMI V  (felt by nearly everyone; some dishes, windows broken; unstable objects overturned).
  Absence of Major Disturbances: The posy doesn\'t indicate any significant damage like fallen objects or structural failures. This suggests the intensity is likely not higher than MMI V.

  Textual Information:
  User’s Report: The user explicitly states feeling the earthquake and using the hashtag #earthquake, confirming the occurrence of seismic activity and its perception by individuals.

  Additional Considerations and Limitations:
  Limited Field of View: The post only shows a portion of the room, restricting a complete assessment of the shaking\'s impact on the environment.
  Building Characteristics: Information about the building\'s construction type (e.g., wood, concrete) and its age would provide valuable context for interpreting the observed effects.
  Ground Motion Data: Access to seismological data from nearby stations would offer quantitative measurements of ground shaking, enabling a more accurate determination of the earthquake\'s intensity at the location.

  Example 2:
  Here is the input data:"""

  text2 = """Output:

  {\"felt_by_user\": \"No\",
   \"class_source\": \"CCTV Footage (YouTube)\",
   \"post_location\": \"Trinidad and Tobago (specific location within the islands not identifiable)\",
   \"post_date_time\": \"August 21, 2018 (exact time not specified)\",
   \"earthquake_location\": \"Carupano, Venezuela\",
   \"earthquake_magnitude\": \"7.3 (provided in video)\",
   \"distance_to_earthquake_epicenter\": \"170 miles (provided in video)\",
   \"shaking_duration\": \"Several seconds, as observed in the video footage.\",
   \"building_type\": \"Grocery store (commercial building)\",
   \"building_materials\": \"Unknown\",
   \"humans_reaction\": \"Panic and fear, evident from their hurried movements and attempts to seek shelter.\",
   \"animals_reaction\": \"Panic and fear, evident from their hurried movements and attempts to seek shelter.\",
   \"furnishing\": \"Grocery store shelves, products, and displays\",
   \"language\": \"Not discernible in the video\",
   \"natural_environment\": \"Not visible in the video\",
   \"video_evidence\": {
     \"Extensive Object Movement\": \"Products falling from shelves, indicating strong ground shaking.\",
     \"Shelves Swaying\": \"Visible swaying and instability of shelves, suggesting significant ground motion.\",
     \"People Running\": \"Individuals rushing towards exits, demonstrating a perception of danger and urgency.\"
   },
   \"audio_evidence\": {
     \"Objects Crashing\": \"Loud and chaotic sounds of items falling and colliding, indicative of intense shaking.\",
     \"People Shouting\": \"Possible sounds of distress or alarm, though not clearly identifiable.\"
   },
   \"textual_evidence\": {
     \"News Reports\": \"Information from UWI Seismic Research Center and USGS reports can be used to corroborate the event and its magnitude.\",
     \"Social Media Posts\": \"Other posts from the region could provide additional context and perspectives on the earthquake\'s impact.\"
   },
   \"mmi_estimation\": \"MMI V or MMI VI\",
   \"mid_mmi_value_numeric\": \"5.5\",
   \"estimation_confidence\": \"0.5\",
   \"visual_observation\": \"The video evidence clearly shows a high degree of ground shaking, with objects falling from shelves and people exhibiting panicked behavior. The swaying of shelves further suggests significant ground motion.\",
   \"auditory_cues\": \"The audio evidence reinforces the intensity of the shaking, with the sounds of crashing objects and potential human distress adding to the overall sense of chaos and urgency.\",
   \"textual_information\": \"While the video itself doesn\'t provide textual information, external sources like news reports and social media posts can be used to gather more context and details about the earthquake.\",
   \"analysis_of_evidences\": \"The combination of visual and auditory cues strongly suggests that the earthquake\'s intensity at this grocery store location was within the MMI V  to MMI VI range. The extensive object movement, swaying shelves, and people\'s panicked reactions all point to a level of shaking that is beyond what would be expected in lower MMI categories.\",
   \"reasoning\": \"The observed effects align with the characteristics of MMI V and VI. The lack of visible structural damage in the video suggests that the intensity might be closer to MMI VI.\"
  }"""

  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 0.5,
      "top_p": 0.95,
  }

  safety_settings = {
      generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
  }

  vertexai.init(project="google.com:quake-eval", location="us-central1")
  model = GenerativeModel("gemini-1.5-pro-preview-0409")
  return model.generate_content(
      [text0, input_data, text1, text2],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )


print("Post ID: ", POST_ID)

mean_estimates = []
try_count = 1
while len(mean_estimates) < 30:
  responses = generate(input_data, distance_km, EARTHQUAKE_NAME_MAGNITUDE)
  responses_text = "".join((response.text for response in responses))
  result = re.search(pattern, responses_text)
  if result:
      float_value = float(result.group(1))
      mean_estimates.append(float_value)

  print(try_count, "try ...", mean_estimates)
  try_count += 1


print('#'*10)
#print("mean_estimates", mean_estimates)
print("median is % s " % (statistics.median(mean_estimates)))
print("mean is % s " % (round(statistics.mean(mean_estimates), 2)))
print("stdev is % s " % (round(statistics.stdev(mean_estimates), 2)))
print('#'*10)

while True:
  responses = generate(input_data, distance_km, EARTHQUAKE_NAME_MAGNITUDE)
  responses_text = "".join((response.text for response in responses))
  if str(statistics.median(mean_estimates)) in responses_text:
    print(responses_text)
    break

Post ID:  us7000ma74_001.jpeg
1 try ... [3.5]
2 try ... [3.5, 3.5]
3 try ... [3.5, 3.5, 2.5]
4 try ... [3.5, 3.5, 2.5, 3.5]
5 try ... [3.5, 3.5, 2.5, 3.5, 3.5]


KeyboardInterrupt: 

In [ ]:
mean_estimates = [6.5, 7.5, 6.5, 7.5, 6.5,
7.5, 7.5, 7.5, 6.5, 6.5,

]
print("mean_estimates", mean_estimates)
print("median is % s " % (statistics.median(mean_estimates)))
print("mean is % s " % (round(statistics.mean(mean_estimates), 2)))
print("stdev is % s " % (round(statistics.stdev(mean_estimates), 2)))

mean_estimates [6.5, 7.5, 6.5, 7.5, 6.5, 7.5, 7.5, 7.5, 6.5, 6.5]
median is 7.0 
mean is 7.0 
stdev is 0.53 


In [ ]:
## second round, fully automated

# def get_response(input_data, distance_to_epicentr_km, earthquake_magnitude):
#   time.sleep(30)

#   text0 = f"""Here is the input data that a user, at {distance_to_epicentr_km} km from the epicenter of a M{earthquake_magnitude} earthquake, reported: """

#   text1 = """You are an earthquake seismologist. Considering the popularity of YouTube and social media platforms like Twitter and TikTok, you intend to use the multimedia content in social media posts shared by people who experienced an earthquake to estimate the intensity of earthquake ground shaking, in the Modified Mercalli Intensity (MMI) Scale, at their locations. Please estimate the intensity of earthquake ground shaking using the provided examples as a reference. Moreover, classify it into two classes of: a) CCTV Footage or b) Social-Media Post/News Interview. Make sure to limit your response to a JSON format containing only the following keys: "felt_by_user", "class_source", "post_location", "post_date_time", "earthquake_location", "earthquake_magnitude", "distance_to_earthquake_epicenter", "shaking_duration", "building_type",  "building_materials", "human_reaction","animals_reaction", "furnishing", "language", "natural_environment", "video_evidence", "audio_evidence", "textual_evidence", "mmi_estimation", "mid_mmi_value_numeric","estimation_confidence", "visual_observation", "auditory_cues", "textual_information", "analysis_of_evidences", "reasoning". Let's walk this through step by step.

#   Example 1:
#   Input:
#   This is an example of input Tweets, posted by a user who felt the M 4.6 earthquake, 3 miles from the epicenter:

#   Output:
#   Available Data:
#   Class Source: social-media post/news interviews (Twitter)
#   Felt by User: No
#   Post’s Location: Imperial, CA
#   Post’s Date and Time: February 12, 2:57 AM
#   Earthquake\'s Location: Not identifiable from post content.
#   Earthquake\'s Magnitude: 4.6 (provided in prompt)
#   Distance to Earthquake Epicenter: 2 km (provided in prompt)
#   Shaking Duration: Not visible in the content.
#   Building Type: Residential building.
#   Building Materials: Wooden Framed.
#   Human\'s Reaction: Not visible in the content.
#   Animal\'s Reaction: Not visible in the content.
#   Furnishings:  Hanging light fixture (chandelier or pendant light), table and chairs, cabinets, and other typical kitchen items.
#   Natural Environment: Not visible.

#   Textual Evidence:
#   Hashtag #earthquake: The user\'s inclusion of \"#earthquake\" explicitly indicates they experienced an earthquake.

#   MMI Scale Estimation:
#   Based on the available evidence, the earthquake intensity at this location is likely within the M range:
#   MMI IV (Light): During the day felt indoors by many, outdoors by few. At night some awakened. Dishes, windows, and doors disturbed; walls make cracking sounds. Sensation like a heavy truck striking building. Standing motor cars rocked noticeably.
#   MMI V (Moderate): Felt by nearly everyone; many awakened. Some Dishes, windows broken. Unstable objects overturned. Pendulum clocks may stop.

#   Reasoning:
#   Visual Observations:
#   Chandelier Swinging: The prominent movement of the chandelier is a clear indicator of ground shaking. Its swinging motion suggests a level of intensity consistent with MMI IV (felt indoors by many, outdoors by few; dishes, windows, doors disturbed; walls make cracking sound) or MMI V  (felt by nearly everyone; some dishes, windows broken; unstable objects overturned).
#   Absence of Major Disturbances: The posy doesn\'t indicate any significant damage like fallen objects or structural failures. This suggests the intensity is likely not higher than MMI V.

#   Textual Information:
#   User’s Report: The user explicitly states feeling the earthquake and using the hashtag #earthquake, confirming the occurrence of seismic activity and its perception by individuals.

#   Additional Considerations and Limitations:
#   Limited Field of View: The post only shows a portion of the room, restricting a complete assessment of the shaking\'s impact on the environment.
#   Building Characteristics: Information about the building\'s construction type (e.g., wood, concrete) and its age would provide valuable context for interpreting the observed effects.
#   Ground Motion Data: Access to seismological data from nearby stations would offer quantitative measurements of ground shaking, enabling a more accurate determination of the earthquake\'s intensity at the location.

#   Example 2:
#   Here is the input data:"""

#   text2 = """Output:

#   {\"felt_by_user\": \"No\",
#    \"class_source\": \"CCTV Footage (YouTube)\",
#    \"post_location\": \"Trinidad and Tobago (specific location within the islands not identifiable)\",
#    \"post_date_time\": \"August 21, 2018 (exact time not specified)\",
#    \"earthquake_location\": \"Carupano, Venezuela\",
#    \"earthquake_magnitude\": \"7.3 (provided in video)\",
#    \"distance_to_earthquake_epicenter\": \"170 miles (provided in video)\",
#    \"shaking_duration\": \"Several seconds, as observed in the video footage.\",
#    \"building_type\": \"Grocery store (commercial building)\",
#    \"building_materials\": \"Unknown\",
#    \"humans_reaction\": \"Panic and fear, evident from their hurried movements and attempts to seek shelter.\",
#    \"animals_reaction\": \"Panic and fear, evident from their hurried movements and attempts to seek shelter.\",
#    \"furnishing\": \"Grocery store shelves, products, and displays\",
#    \"language\": \"Not discernible in the video\",
#    \"natural_environment\": \"Not visible in the video\",
#    \"video_evidence\": {
#      \"Extensive Object Movement\": \"Products falling from shelves, indicating strong ground shaking.\",
#      \"Shelves Swaying\": \"Visible swaying and instability of shelves, suggesting significant ground motion.\",
#      \"People Running\": \"Individuals rushing towards exits, demonstrating a perception of danger and urgency.\"
#    },
#    \"audio_evidence\": {
#      \"Objects Crashing\": \"Loud and chaotic sounds of items falling and colliding, indicative of intense shaking.\",
#      \"People Shouting\": \"Possible sounds of distress or alarm, though not clearly identifiable.\"
#    },
#    \"textual_evidence\": {
#      \"News Reports\": \"Information from UWI Seismic Research Center and USGS reports can be used to corroborate the event and its magnitude.\",
#      \"Social Media Posts\": \"Other posts from the region could provide additional context and perspectives on the earthquake\'s impact.\"
#    },
#    \"mmi_estimation\": \"MMI V or MMI VI\",
#    \"mid_mmi_value_numeric\": \"5.5\",
#    \"estimation_confidence\": \"0.5\",
#    \"visual_observation\": \"The video evidence clearly shows a high degree of ground shaking, with objects falling from shelves and people exhibiting panicked behavior. The swaying of shelves further suggests significant ground motion.\",
#    \"auditory_cues\": \"The audio evidence reinforces the intensity of the shaking, with the sounds of crashing objects and potential human distress adding to the overall sense of chaos and urgency.\",
#    \"textual_information\": \"While the video itself doesn\'t provide textual information, external sources like news reports and social media posts can be used to gather more context and details about the earthquake.\",
#    \"analysis_of_evidences\": \"The combination of visual and auditory cues strongly suggests that the earthquake\'s intensity at this grocery store location was within the MMI V  to MMI VI range. The extensive object movement, swaying shelves, and people\'s panicked reactions all point to a level of shaking that is beyond what would be expected in lower MMI categories.\",
#    \"reasoning\": \"The observed effects align with the characteristics of MMI V and VI. The lack of visible structural damage in the video suggests that the intensity might be closer to MMI VI.\"
#   }"""

#   generation_config = {
#       "max_output_tokens": 8192,
#       "temperature": 0.5,
#       "top_p": 0.95,
#   }

#   safety_settings = {
#       generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#       generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#       generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#       generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#   }

#   vertexai.init(project="google.com:quake-eval", location="us-central1")
#   model = GenerativeModel("gemini-1.5-pro-preview-0409")
#   return model.generate_content(
#       [text0, input_data, text1, text2],
#       generation_config=generation_config,
#       safety_settings=safety_settings,
#       stream=True,
#   )



event_mag_epicenter_by_event_id = {
            "ok2024cish":{"mag": 5.1, "lat_lng":(35.534, -96.764)},
            "us7000ma74":{"mag": 4.8, "lat_lng":(40.696, -74.760)},
            "us7000m9g4":{"mag": 7.4, "lat_lng":(23.832, 121.604)},
            "nc73886731":{"mag": 5.1, "lat_lng":(40.204, -121.110)},
            "us6000m0xl":{"mag": 7.5, "lat_lng":(37.487, 137.271)},
            "official20110311054624120":{"mag": 9.1, "lat_lng":(38.297, 142.373)},
            "ci40664632":{"mag": 4.6, "lat_lng":(34.058, 118.908)},
            }

# 'us7000ma74_029.mp4': 63.35, 'us7000ma74_001.jpeg': 50.87, 'us7000ma74_000.jpeg': 64.3, 'us7000ma74_002.jpeg': 95, 'us7000ma74_003.jpeg': 79.99, 'us7000ma74_004.jpeg': 73.16, 'us7000ma74_005.jpeg': 67.16, 'us7000ma74_006.jpeg': 69.24, 'us7000ma74_007.jpeg': 62.71,
dist_by_post_id = {'us7000ma74_017.mp4': 11.92, 'us7000ma74_018.mp4': 52.27, 'us7000ma74_019.mp4': 37.59, 'us7000ma74_008.jpeg': 161.89, 'us7000ma74_030.mp4': 67.24, 'us7000ma74_035.mp4': 65.66, 'us7000ma74_032.mp4': 59.6, 'us7000ma74_033.mp4': 32.1869, 'us7000ma74_034.mp4': 128.05, 'us7000ma74_037.mp4': 42.83, 'us7000ma74_020.mp4': 63.75, 'us7000ma74_021.mp4': 62.68, 'us7000ma74_022.mp4': 44.53, 'us7000ma74_023.mp4': 33.96, 'us7000ma74_025.mp4': 39.74, 'us7000ma74_026.mp4': 63.41, 'us7000ma74_010.mp4': 64.64, 'us7000ma74_011.mp4': 63.2, 'us7000ma74_013.mp4': 40.98, 'us7000ma74_014.mp4': 91.64, 'us7000ma74_058.mp4': 50.46, 'us7000ma74_057.mp4': 78.76, 'us7000ma74_050.mp4': 252.16, 'us7000ma74_051.mp4': 56.54, 'us7000ma74_052.mp4': 68.13, 'us7000ma74_053.mp4': 69.89, 'us7000ma74_054.mp4': 77.5, 'us7000ma74_055.mp4': 63.77, 'us7000ma74_056.mp4': 57, 'us7000ma74_038.mp4': 44.53, 'us7000ma74_040.mp4': 71.92, 'us7000ma74_041.mp4': 63, 'us7000ma74_042.mp4': 63.75, 'us7000ma74_043.mp4': 78.4, 'us7000ma74_046.mp4': 63.85, 'us7000ma74_047.mp4': 70, 'ok2024cish_000.jpeg': 90, 'ok2024cish_001.jpeg': 64.97, 'ok2024cish_002.jpeg': 92.31, 'ok2024cish_005.mp4': 78.95, 'ok2024cish_006.mp4': 78.95, 'ok2024cish_007.mp4': 105.12, 'ok2024cish_008.mp4': 98.4, 'ok2024cish_009.mp4': 98.09, 'ok2024cish_010.mp4': 68.56, 'ok2024cish_011.mp4': 68.56, 'ci39645386_001.mp4': 10.91, 'ci39645386_002.mp4': 10.91, 'ci40664632_001.mp4': 13.55, 'ci40664632_002.mp4': 61.42, 'nc73886731_001.mp4': 5.13, 'nc73886731_002.mp4': 6.0, 'nc73886731_003.mp4': 8.0, 'us7000m9g4_001.mp4': 17.72, 'us7000m9g4_003.mp4': 133.21, 'us7000m9g4_004.mp4': 17.72, 'us7000m9g4_005.mp4': 134.55, 'us7000m9g4_007.mp4': 17.72, 'us7000m9g4_008.mp4': 17.72, 'us7000m9g4_010.mp4': 17, 'us6000m0xl_009.mp4': 96.89, 'us6000m0xl_007.mp4': 68.42, 'us6000m0xl_006.mp4': 5.2, 'us6000m0xl_005.mp4': 70.0, 'us6000m0xl_004.mp4': 70.0, 'us6000m0xl_003.mp4': 93.36, 'us6000m0xl_002.mp4': 68.42, 'us6000m0xl_001.mp4': 115.1, 'official20110311054624120_001.mp4': 131.38, 'official20110311054624120_002.mp4': 131.38, 'official20110311054624120_003.mp4': 131.38, 'official20110311054624120_004.mp4': 131.38, 'official20110311054624120_005.mp4': 131.38, 'official20110311054624120_007.mp4': 131.38, 'official20110311054624120_008.mp4': 131.38}


results_by_post_id = {}
p_count = 1
for post_id in dist_by_post_id:
  post_distance_to_earthquake = dist_by_post_id[post_id]
  event_mag = event_mag_epicenter_by_event_id[post_id.split('_')[0]]["mag"]

  DATASET_PATH = "quake_gemini_project/gemini_project_dataset"
  EVENT_DIRECTORY = "all_posts"

  if post_id.split(".")[1] == "mp4":
    input_data = Part.from_uri(
        mime_type="video/mp4",
        uri=f"gs://{DATASET_PATH}/{EVENT_DIRECTORY}/{post_id}")
  elif post_id.split(".")[1] == "jpeg":
    input_data = Part.from_uri(
        mime_type="image/jpeg",
        uri=f"gs://{DATASET_PATH}/{EVENT_DIRECTORY}/{post_id}")
    get_response

  print("Post ID: ", post_id, "(", p_count, "out of: ", len(dist_by_post_id),")")
  p_count +=1

  mean_estimates = []
  try_count = 1
  while len(mean_estimates) < 10:
    responses = get_response(input_data, post_distance_to_earthquake, event_mag)
    responses_text = "".join((response.text for response in responses))
    result = re.search(pattern, responses_text)
    if result:
        float_value = float(result.group(1))
        mean_estimates.append(float_value)
    #print(try_count, "try ...", mean_estimates)
    try_count += 1


  print('#'*10)
  print("mean_estimates", mean_estimates)
  print("median is % s " % (statistics.median(mean_estimates)))
  print("mean is % s " % (round(statistics.mean(mean_estimates), 2)))
  print("stdev is % s " % (round(statistics.stdev(mean_estimates), 2)))
  print('#'*10)
  results_by_post_id[post_id] = ((statistics.median(mean_estimates)),
                           (round(statistics.mean(mean_estimates), 2)),
                           (round(statistics.stdev(mean_estimates), 2))
                           )
storage_client = google.cloud.storage.Client()
bucket = storage_client.bucket('quake_gemini_project')
blob = bucket.blob('results_by_post_id.pkl')

with blob.open('wb') as f:
    pickle.dump(results_by_post_id, f)

Post ID:  us7000ma74_017.mp4 ( 1 out of:  75 )
##########
mean_estimates [3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5]
median is 3.5 
mean is 3.5 
stdev is 0.0 
##########
Post ID:  us7000ma74_018.mp4 ( 2 out of:  75 )


AttributeError: Content has no parts.